In [413]:
#Solve Karnaugh Maps using Q-map Solver Scraper.

In [414]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
import time

In [536]:
truth_table = ["""1	1	0	0	0	0	0
1	1	1	1	0	1	0
0	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
1	1	1	1	1	1	1
0	1	0	1	0	1	0
0	0	0	1	0	0	0
0	1	1	0	0	0	1
1	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
0	1	1	0	0	0	0
1	1	1	1	1	1	1
1	1	1	1	1	1	1
1	1	1	1	1	1	1""","""1	1	1	1	1	1	1
1	1	0	0	0	0	0
1	1	1	1	0	1	0
0	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
1	1	1	1	1	1	1
0	1	0	1	0	1	0
0	0	0	1	0	0	0
0	1	1	0	0	0	1
1	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
0	1	1	0	0	0	0
1	1	1	1	1	1	1
1	1	1	1	1	1	1""","""1	1	1	1	1	1	1
1	1	1	1	1	1	1
1	1	0	0	0	0	0
1	1	1	1	0	1	0
0	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
1	1	1	1	1	1	1
0	1	0	1	0	1	0
0	0	0	1	0	0	0
0	1	1	0	0	0	1
1	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
0	1	1	0	0	0	0
1	1	1	1	1	1	1""","""1	1	1	1	1	1	1
1	1	1	1	1	1	1
1	1	1	1	1	1	1
1	1	0	0	0	0	0
1	1	1	1	0	1	0
0	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
1	1	1	1	1	1	1
0	1	0	1	0	1	0
0	0	0	1	0	0	0
0	1	1	0	0	0	1
1	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
0	1	1	0	0	0	0"""]

LED_truth_table = ["""0	0	0	1
0	0	1	0
0	0	1	1
0	1	0	0
0	1	0	1
0	1	1	0
0	1	1	1
1	0	0	0
1	0	0	1
1	0	1	0
1	0	1	1
1	1	0	0
1	1	0	1
1	1	1	0
1	1	1	1
0	0	0	0"""]

In [537]:
def raw_one_data(truth_table):
    """Converts list() of truth table to individual set. Does for one table only."""
    truth_table = truth_table.replace("\t", ",")
#     if(truth_table.index("X") != -1):
    truth_table = truth_table.replace("x", "2")
    truth_table = np.array(truth_table.split("\n")) 
    new_truth_table = []
    for i in range(len(truth_table)):
        one_val = truth_table[i].split(",")
        one_row = []
        for app in one_val:
            one_row.append(int(app))
        new_truth_table.append(one_row)
    return np.array(new_truth_table)

In [538]:
def process_data(table):
    """
    Process Each truth table.
    Returns a list of processed truth tables.
    """
    processed_truth_table = []
    for one_table in table:
        processed_truth_table.append(raw_one_data(one_table))
    return np.array(processed_truth_table)

In [539]:
def data_launch(br):
    """
    launches the driver session's k-map solver webpage.
    """
    br.get("http://www.32x8.com/var4.html") 
    table = br.find_elements_by_tag_name("table")[-1]
    rows = table.find_element_by_tag_name("tbody")
    rows = rows.find_elements_by_tag_name("tr")[2:]
    return rows

In [540]:
import chromedriver_binary ##--> Change the Chrome Version Build to v.78 for executable path.
br = webdriver.Chrome(executable_path=r'C:/Users/qasim/Downloads/chromedriver_win32_v78/chromedriver.exe')
rows = data_launch(br)

In [541]:
def insert_data(elem, data):
    """
    Inserts data from the np.array() of k-maps into the table
    on chrome-driver session.
    Submits the page.
    Parameters:
    1. elem: The set of np.array() elements with input field.
    2. data: np.array() 1x16 array of data.
    """
    
    insert = elem[0:-1]
    for value, radio_web in zip(data, insert):
        row = radio_web.find_elements_by_tag_name("input")
        for i in range(len(row)):
            if(i == value):
                row[i].click()
    elem[-1].find_elements_by_tag_name("input")[-1].click()

In [542]:
def one_hex_input(elem, data):
    """
    Returns a list of K-map equations for each individual LED display.
    Parameters:
    1. elem: np.array() row object.
    2. data: 16x7 vector of k-map truth table data.
    Returns:
    1. List: np.array() object of shape => 7x1, set of equations.
    """
    num = 0
    equations = []
    dataframes = []
    new_truth_table = data.T
    for x in new_truth_table:
        insert_data(elem, x)
        sop_elem = br.find_element_by_css_selector("div#sop").find_elements_by_css_selector("div")[5].text 

        sop_elem = sop_elem.replace("A", "Y₃")
        sop_elem = sop_elem.replace("B", "Y₂")
        sop_elem = sop_elem.replace("C", "Y₁")
        sop_elem = sop_elem.replace("D", "Y₀")
        sop_elem = sop_elem.replace("y", "D[" + str(num) + "]")

        df = br.find_element_by_css_selector("div#sop").find_elements_by_css_selector("div")[2].text  
        grid = []
        df = df.split("\n")[2:]
        for spl in df:
            grid.append(spl.split(" ")[1:])
        equations.append(sop_elem)
        df = pd.DataFrame(grid, index=['00', '01', '11', '10'], columns=['00', '01', '11', '10'])
        dataframes.append(df)
        elem = data_launch(br)
        num += 1
    return np.array(equations), dataframes

In [543]:
def create_func(rows, new_truth_table):
    k_maps = []
    grids = []
    for tables in new_truth_table:
        km, gr = one_hex_input(rows, tables)
        k_maps.append(km)
        grids.append(gr)
        rows = data_launch(br)
    return np.array(k_maps), grids

In [544]:
new_truth_table = process_data(truth_table)
k_maps, grids = create_func(rows, new_truth_table)

In [565]:
k_maps

array([["D[0] = Y₃'Y₁' + Y₂'Y₀ + Y₁'Y₀ + Y₃Y₁",
        "D[1] = Y₁'Y₀' + Y₂Y₁' + Y₂Y₀' + Y₃Y₁ + Y₃'Y₂'Y₀",
        "D[2] = Y₃Y₀' + Y₃Y₂ + Y₃'Y₂'Y₀ + Y₃'Y₁'Y₀",
        "D[3] = Y₀ + Y₁ + Y₃'Y₂", "D[4] = Y₂Y₁'Y₀ + Y₃Y₂Y₁",
        "D[5] = Y₃'Y₁'Y₀ + Y₃'Y₂Y₁' + Y₂Y₁'Y₀ + Y₂Y₁Y₀' + Y₃Y₁Y₀",
        "D[6] = Y₃Y₂Y₁ + Y₃Y₂'Y₀' + Y₂Y₁'Y₀ + Y₃'Y₂'Y₁Y₀"],
       ["D[0] = Y₃'Y₁' + Y₃'Y₀' + Y₂Y₀' + Y₃Y₁",
        "D[1] = Y₂ + Y₃'Y₁' + Y₃'Y₀' + Y₃Y₀",
        "D[2] = Y₃'Y₀' + Y₃Y₀ + Y₂Y₁Y₀'", "D[3] = Y₀' + Y₁ + Y₃'Y₂",
        "D[4] = Y₂Y₁Y₀' + Y₃Y₂Y₁ + Y₃'Y₂'Y₁'Y₀'",
        "D[5] = Y₂Y₁ + Y₃'Y₂'Y₀' + Y₃'Y₂Y₀ + Y₃Y₂Y₀'",
        "D[6] = Y₃'Y₁'Y₀' + Y₂Y₁Y₀' + Y₃Y₂'Y₀ + Y₃Y₁Y₀"],
       ["D[0] = Y₃'Y₂' + Y₃'Y₀ + Y₃'Y₁ + Y₁Y₀ + Y₃Y₂Y₁'",
        "D[1] = Y₃'Y₂' + Y₃'Y₁ + Y₂Y₀ + Y₃Y₀'",
        "D[2] = Y₃'Y₀ + Y₃'Y₂'Y₁' + Y₂Y₁Y₀ + Y₃Y₁Y₀' + Y₃Y₂Y₀'",
        "D[3] = Y₁' + Y₀ + Y₃'Y₂", "D[4] = Y₃'Y₂'Y₁' + Y₂Y₁Y₀",
        "D[5] = Y₂'Y₁'Y₀' + Y₃'Y₂'Y₀ + Y₃'Y₂Y₁ + Y₃Y₂Y₀",
        "D[6] = Y₃'Y₂'Y₁' + Y₃'Y

In [545]:
LED_rows = data_launch(br)
new_LED_truth_table = process_data(LED_truth_table)
led_kmaps, led_grids = create_func(LED_rows, new_LED_truth_table)

In [546]:
display(grids[3][4]),print("\n"), display(grids[3][5]), print("\n"), display(grids[3][6])

,00,01,11,10
00,1,1,0,1
01,0,0,0,0
11,0,0,0,0
10,1,0,0,0


,00,01,11,10
00,1,1,0,1
01,1,0,1,0
11,0,0,0,1
10,1,1,0,0


,00,01,11,10
00,1,1,0,1
01,0,0,0,1
11,0,1,0,0
10,1,0,1,0


(None, None, None, None, None)

In [547]:
grids[3][6]

,00,01,11,10
00,1,1,0,1
01,0,0,0,1
11,0,1,0,0
10,1,0,1,0


In [567]:
led_kmaps

array([["D[0] = Y₃Y₂' + Y₃Y₁' + Y₃Y₀' + Y₃'Y₂Y₁Y₀",
        "D[1] = Y₂Y₁' + Y₂Y₀' + Y₂'Y₁Y₀", "D[2] = Y₁'Y₀ + Y₁Y₀'",
        "D[3] = Y₀'"]], dtype='<U40')

In [549]:
grids[0][0]

,00,01,11,10
00,1,1,1,0
01,1,1,0,0
11,0,1,1,1
10,0,1,1,1


In [550]:
# LED_truth_table.replace("X", "2")
str(LED_truth_table).replace("X", "2")

"['0\\t0\\t0\\t1\\n0\\t0\\t1\\t0\\n0\\t0\\t1\\t1\\n0\\t1\\t0\\t0\\n0\\t1\\t0\\t1\\n0\\t1\\t1\\t0\\n0\\t1\\t1\\t1\\n1\\t0\\t0\\t0\\n1\\t0\\t0\\t1\\n1\\t0\\t1\\t0\\n1\\t0\\t1\\t1\\n1\\t1\\t0\\t0\\n1\\t1\\t0\\t1\\n1\\t1\\t1\\t0\\n1\\t1\\t1\\t1\\n0\\t0\\t0\\t0']"

In [551]:
k_maps

array([["D[0] = Y₃'Y₁' + Y₂'Y₀ + Y₁'Y₀ + Y₃Y₁",
        "D[1] = Y₁'Y₀' + Y₂Y₁' + Y₂Y₀' + Y₃Y₁ + Y₃'Y₂'Y₀",
        "D[2] = Y₃Y₀' + Y₃Y₂ + Y₃'Y₂'Y₀ + Y₃'Y₁'Y₀",
        "D[3] = Y₀ + Y₁ + Y₃'Y₂", "D[4] = Y₂Y₁'Y₀ + Y₃Y₂Y₁",
        "D[5] = Y₃'Y₁'Y₀ + Y₃'Y₂Y₁' + Y₂Y₁'Y₀ + Y₂Y₁Y₀' + Y₃Y₁Y₀",
        "D[6] = Y₃Y₂Y₁ + Y₃Y₂'Y₀' + Y₂Y₁'Y₀ + Y₃'Y₂'Y₁Y₀"],
       ["D[0] = Y₃'Y₁' + Y₃'Y₀' + Y₂Y₀' + Y₃Y₁",
        "D[1] = Y₂ + Y₃'Y₁' + Y₃'Y₀' + Y₃Y₀",
        "D[2] = Y₃'Y₀' + Y₃Y₀ + Y₂Y₁Y₀'", "D[3] = Y₀' + Y₁ + Y₃'Y₂",
        "D[4] = Y₂Y₁Y₀' + Y₃Y₂Y₁ + Y₃'Y₂'Y₁'Y₀'",
        "D[5] = Y₂Y₁ + Y₃'Y₂'Y₀' + Y₃'Y₂Y₀ + Y₃Y₂Y₀'",
        "D[6] = Y₃'Y₁'Y₀' + Y₂Y₁Y₀' + Y₃Y₂'Y₀ + Y₃Y₁Y₀"],
       ["D[0] = Y₃'Y₂' + Y₃'Y₀ + Y₃'Y₁ + Y₁Y₀ + Y₃Y₂Y₁'",
        "D[1] = Y₃'Y₂' + Y₃'Y₁ + Y₂Y₀ + Y₃Y₀'",
        "D[2] = Y₃'Y₀ + Y₃'Y₂'Y₁' + Y₂Y₁Y₀ + Y₃Y₁Y₀' + Y₃Y₂Y₀'",
        "D[3] = Y₁' + Y₀ + Y₃'Y₂", "D[4] = Y₃'Y₂'Y₁' + Y₂Y₁Y₀",
        "D[5] = Y₂'Y₁'Y₀' + Y₃'Y₂'Y₀ + Y₃'Y₂Y₁ + Y₃Y₂Y₀",
        "D[6] = Y₃'Y₂'Y₁' + Y₃'Y

In [552]:
led_kmaps

array([["D[0] = Y₃Y₂' + Y₃Y₁' + Y₃Y₀' + Y₃'Y₂Y₁Y₀",
        "D[1] = Y₂Y₁' + Y₂Y₀' + Y₂'Y₁Y₀", "D[2] = Y₁'Y₀ + Y₁Y₀'",
        "D[3] = Y₀'"]], dtype='<U40')

In [553]:
#32 K_maps with their tables in less than 45 seconds.

In [554]:
led_grids[0][0]

,00,01,11,10
00,0,0,0,0
01,0,0,1,0
11,1,1,0,1
10,1,1,1,1


In [585]:
print("HEX Display K-Maps. Number = " + str(len(k_maps)*len(k_maps[0])))
i = 0
for maps in k_maps:
    print("HEX Display Number: " + str(i))
    for mx in maps:
        print("\n",mx)
    print("--------------------------------------")
    i += 1

HEX Display K-Maps. Number = 28
HEX Display Number: 0

 D[0] = Y₃'Y₁' + Y₂'Y₀ + Y₁'Y₀ + Y₃Y₁

 D[1] = Y₁'Y₀' + Y₂Y₁' + Y₂Y₀' + Y₃Y₁ + Y₃'Y₂'Y₀

 D[2] = Y₃Y₀' + Y₃Y₂ + Y₃'Y₂'Y₀ + Y₃'Y₁'Y₀

 D[3] = Y₀ + Y₁ + Y₃'Y₂

 D[4] = Y₂Y₁'Y₀ + Y₃Y₂Y₁

 D[5] = Y₃'Y₁'Y₀ + Y₃'Y₂Y₁' + Y₂Y₁'Y₀ + Y₂Y₁Y₀' + Y₃Y₁Y₀

 D[6] = Y₃Y₂Y₁ + Y₃Y₂'Y₀' + Y₂Y₁'Y₀ + Y₃'Y₂'Y₁Y₀
--------------------------------------
HEX Display Number: 1

 D[0] = Y₃'Y₁' + Y₃'Y₀' + Y₂Y₀' + Y₃Y₁

 D[1] = Y₂ + Y₃'Y₁' + Y₃'Y₀' + Y₃Y₀

 D[2] = Y₃'Y₀' + Y₃Y₀ + Y₂Y₁Y₀'

 D[3] = Y₀' + Y₁ + Y₃'Y₂

 D[4] = Y₂Y₁Y₀' + Y₃Y₂Y₁ + Y₃'Y₂'Y₁'Y₀'

 D[5] = Y₂Y₁ + Y₃'Y₂'Y₀' + Y₃'Y₂Y₀ + Y₃Y₂Y₀'

 D[6] = Y₃'Y₁'Y₀' + Y₂Y₁Y₀' + Y₃Y₂'Y₀ + Y₃Y₁Y₀
--------------------------------------
HEX Display Number: 2

 D[0] = Y₃'Y₂' + Y₃'Y₀ + Y₃'Y₁ + Y₁Y₀ + Y₃Y₂Y₁'

 D[1] = Y₃'Y₂' + Y₃'Y₁ + Y₂Y₀ + Y₃Y₀'

 D[2] = Y₃'Y₀ + Y₃'Y₂'Y₁' + Y₂Y₁Y₀ + Y₃Y₁Y₀' + Y₃Y₂Y₀'

 D[3] = Y₁' + Y₀ + Y₃'Y₂

 D[4] = Y₃'Y₂'Y₁' + Y₂Y₁Y₀

 D[5] = Y₂'Y₁'Y₀' + Y₃'Y₂'Y₀ + Y₃'Y₂Y₁ + Y₃Y₂Y₀

 D[6] = Y₃'Y₂

In [556]:
print("LED Display K-Maps. Number = " + str(len(led_kmaps[0])))
for lmap in reversed(led_kmaps[0]):
    print("\n",lmap)

LED Display K-Maps. Number = 4

 D[3] = Y₀'

 D[2] = Y₁'Y₀ + Y₁Y₀'

 D[1] = Y₂Y₁' + Y₂Y₀' + Y₂'Y₁Y₀

 D[0] = Y₃Y₂' + Y₃Y₁' + Y₃Y₀' + Y₃'Y₂Y₁Y₀


In [557]:
grids[3][6]

,00,01,11,10
00,1,1,0,1
01,0,0,0,1
11,0,1,0,0
10,1,0,1,0


In [558]:
led_kmaps

array([["D[0] = Y₃Y₂' + Y₃Y₁' + Y₃Y₀' + Y₃'Y₂Y₁Y₀",
        "D[1] = Y₂Y₁' + Y₂Y₀' + Y₂'Y₁Y₀", "D[2] = Y₁'Y₀ + Y₁Y₀'",
        "D[3] = Y₀'"]], dtype='<U40')

In [568]:
def transistor_count_one(equation):
    """Finds the transistor count for one assign statement."""
    test_2 = equation
    test_2 = test_2.split("= ")[1]
    # --> Standard
    test_2 = test_2.replace("Y₀", "D")
    test_2 = test_2.replace("Y₁", "C")
    test_2 = test_2.replace("Y₂", "B")
    test_2 = test_2.replace("Y₃", "A")
    test_2 = test_2.replace("A'", "a")
    test_2 = test_2.replace("B'", "b")
    test_2 = test_2.replace("C'", "c")
    test_2 = test_2.replace("D'", "d")

    delimiters = test_2.split(" + ")
    a = 0
    b = 0
    c = 0
    d = 0
    for char in test_2:
        if(char == 'a'):
            a += 1
        if(char == 'b'):
            b += 1
        if(char == 'c'):
            c += 1
        if(char == 'd'):
            d += 1

    inverter_count = 0
    if(a > 0):
        inverter_count += 2
    if(b > 0):
        inverter_count += 2
    if(c > 0):
        inverter_count += 2
    if(d > 0):
        inverter_count += 2    

    parent = (len(delimiters)-2)*2 + 4
    count_ind = 0
    for ch in delimiters:
        count_ind += (len(ch)-2)*2 + 4
    total_transistor_count = count_ind + parent + inverter_count
    return total_transistor_count

In [572]:
def cumulative_transistor_count(data):
    """Finds the cumulative transistor count for all equations."""
    spawn = []
    tots_child = []
    total = 0
    for i in range(len(data)):
        child = []
        for j in range(len(data[i])):
            uno = transistor_count_one(data[i][j])
            child.append(uno)
        spawn.append(child)
        tots_child.append(sum(child))
        total += sum(child)
    return spawn, tots_child, total

In [573]:
num, total_ind, cum_full = cumulative_transistor_count(k_maps)

In [574]:
num, total_ind, cum_full

([[30, 40, 36, 16, 18, 46, 42],
  [30, 28, 24, 18, 34, 36, 40],
  [38, 30, 46, 18, 22, 40, 52],
  [40, 38, 46, 20, 32, 50, 52]],
 [228, 210, 246, 278],
 962)

In [563]:
transistor_count_one(led_kmaps[0][3])

6

In [564]:
led_kmaps

array([["D[0] = Y₃Y₂' + Y₃Y₁' + Y₃Y₀' + Y₃'Y₂Y₁Y₀",
        "D[1] = Y₂Y₁' + Y₂Y₀' + Y₂'Y₁Y₀", "D[2] = Y₁'Y₀ + Y₁Y₀'",
        "D[3] = Y₀'"]], dtype='<U40')

In [582]:
grids[3][5]

,00,01,11,10
00,1,1,0,1
01,1,0,1,0
11,0,0,0,1
10,1,1,0,0


In [584]:
k_maps[0]

array(["D[0] = Y₃'Y₁' + Y₂'Y₀ + Y₁'Y₀ + Y₃Y₁",
       "D[1] = Y₁'Y₀' + Y₂Y₁' + Y₂Y₀' + Y₃Y₁ + Y₃'Y₂'Y₀",
       "D[2] = Y₃Y₀' + Y₃Y₂ + Y₃'Y₂'Y₀ + Y₃'Y₁'Y₀",
       "D[3] = Y₀ + Y₁ + Y₃'Y₂", "D[4] = Y₂Y₁'Y₀ + Y₃Y₂Y₁",
       "D[5] = Y₃'Y₁'Y₀ + Y₃'Y₂Y₁' + Y₂Y₁'Y₀ + Y₂Y₁Y₀' + Y₃Y₁Y₀",
       "D[6] = Y₃Y₂Y₁ + Y₃Y₂'Y₀' + Y₂Y₁'Y₀ + Y₃'Y₂'Y₁Y₀"], dtype='<U63')